In [ ]:
import requests # type: ignore
import pandas as pd # type: ignore
import psycopg2 # type: ignore
from psycopg2 import sql # type: ignore
from tqdm import tqdm # type: ignore
from datetime import datetime, timedelta
import random
import time
from util import get_init_period_trade_cal, db_params, \
    create_table, get_all_codes, sleeper, \
    get_ts, get_trade_cal_from_postgres

In [46]:
year, month, day = 2025, 9, 5
for day_of_month in tqdm(range(1, day+1), desc=f"tpex price update for {year}-{month}-{day}"):
    url = f"https://www.tpex.org.tw/www/zh-tw/afterTrading/dailyQuotes?date={year}%2F{month}%2F{day_of_month}&response=json"
    res = requests.get(url)
    data = res.json()['tables'][0]['data']
    data_copy = data.copy()
    da = res.json()['date']

tpex price update for 2025-9-5: 100%|██████████| 5/5 [00:04<00:00,  1.20it/s]


In [52]:
formatted_da = "20250905"
uri = f"https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date={formatted_da}&selectType=ALL&response=json"
res = requests.get(uri)
data = res.json()['data']

In [ ]:
def build_twse_maincode_from_postgres_stock_yield_pe_pb():
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute("""
    select distinct code, cname from public.stock_yield_pe_pb;
    """)
    data = [i for i in cursor.fetchall()]

    sql_str = ""
    for i in data:
        sql_str += f"""
        insert into public.maincode (code, cname, exchange) values ('{i[0]}', '{i[1]}', 'twse') on conflict (code) do nothing;
        """
    cursor.execute(sql_str)
    conn.commit()

[('9939', '宏全'),
 ('1736', '喬山'),
 ('1504', '東元'),
 ('2812', '台中銀'),
 ('2337', '旺宏'),
 ('4306', '炎洲'),
 ('2301', '光寶科'),
 ('2420', '新巨'),
 ('1617', '榮星'),
 ('1452', '宏益'),
 ('3059', '華晶科'),
 ('2701', '萬企'),
 ('1455', '集盛'),
 ('9924', '福興'),
 ('2359', '所羅門'),
 ('2347', '聯強'),
 ('3045', '台灣大'),
 ('3406', '玉晶光'),
 ('1453', '大將'),
 ('2482', '連宇'),
 ('2331', '精英'),
 ('1337', '再生-KY'),
 ('6702', '興航'),
 ('1539', '巨庭'),
 ('4960', '誠美材'),
 ('1524', '耿鼎'),
 ('6431', '光麗-KY'),
 ('8429', '金麗-KY'),
 ('3094', '聯傑'),
 ('2888', '新光金'),
 ('1735', '日勝化'),
 ('1312', '國喬'),
 ('3019', '亞光'),
 ('3665', '貿聯-KY'),
 ('3021', '鴻名'),
 ('6504', '南六'),
 ('5531', '鄉林'),
 ('4746', '台耀'),
 ('6139', '亞翔'),
 ('1506', '正道'),
 ('3004', '豐達科'),
 ('6164', '華興'),
 ('1710', '東聯'),
 ('1715', '萬洲'),
 ('3419', '譁裕'),
 ('1527', '鑽全'),
 ('1477', '聚陽'),
 ('2903', '遠百'),
 ('2033', '佳大'),
 ('2727', '王品'),
 ('2038', '海光'),
 ('3535', '晶彩科'),
 ('5871', '中租-KY'),
 ('6283', '淳安'),
 ('6235', '華孚'),
 ('1906', '寶隆'),
 ('3669', '圓展'),
 ('14

In [65]:
res.json()

{'stat': '參數輸入錯誤'}

In [76]:
trade_cal = get_init_period_trade_cal('turnover', t_='postgres')
print(trade_cal)

UndefinedTable: relation "public.turnover" does not exist
LINE 2:                 select max(da) from public.turnover;
                                            ^


In [ ]:
# tpex turnover

year, month, day = 2015, 9, 5
'''
https://www.tpex.org.tw/www/zh-tw/afterTrading/turnover?date=2025%2F09%2F04&response=json
https://www.tpex.org.tw/zh-tw/mainboard/trading/historical/turnover-ratio/day.html
'''

url = f"https://www.tpex.org.tw/www/zh-tw/afterTrading/turnover?type=Daily&date={year}%2F{month}%2F{day}&response=json"
res = requests.get(url)
data = res.json()['tables'][0]['data']
data_copy = data.copy()

da = res.json()['date']
sql_str = ""
for i in data_copy:
    i[3] = i[3].replace(',', '')
    i[4] = i[4].replace(',', '')
    i[5] = i[5].replace(',', '')
    sql_str += f"""INSERT INTO public.turnover (da, code, cname, shares_volume, shares_issued, turnover)
    VALUES ('{da}', '{i[1]}', '{i[2]}', '{i[3]}', '{i[4]}') on conflict (da, code) do nothing;"""

cursor.execute(sql_str)
conn.commit()

[['1', '6418', '詠昇', '11,187,048', '34,522,000', '32.40'],
 ['2', '3163', '波若威', '25,641,546', '80,540,741', '31.83'],
 ['3', '4303', '信立', '25,788,789', '94,605,317', '27.25'],
 ['4', '8358', '金居開發', '66,070,267', '252,588,000', '26.15'],
 ['5', '3323', '加百裕', '22,725,380', '93,955,415', '24.18'],
 ['6', '4541', '晟田', '16,125,189', '67,647,068', '23.83'],
 ['7', '5309', '系統電子', '44,961,422', '191,969,576', '23.42'],
 ['8', '5314', '世紀*', '59,758,165', '292,000,000', '20.46'],
 ['9', '1815', '富喬', '101,383,299', '520,886,254', '19.46'],
 ['10', '3707', '漢磊', '70,746,935', '383,222,653', '18.46'],
 ['11', '5439', '高技企業', '14,715,004', '92,975,752', '15.82'],
 ['12', '8088', '品安科技', '8,993,465', '60,921,749', '14.76'],
 ['13', '6761', '穩得', '5,072,307', '34,955,867', '14.51'],
 ['14', '7402', '邑錡', '4,018,431', '35,229,421', '11.40'],
 ['15', '4714', '永捷', '18,642,421', '178,088,188', '10.46'],
 ['16', '6217', '中國探針', '12,271,761', '117,210,018', '10.46'],
 ['17', '1295', '生合', '3,387,60